## Note

This notebook builds on ´Compare Salinometer vs CTD´ -> **Run that one first and this notebook to run on that kernel**!

I.e. go to *Kernel* - *Change kernel* - *Use kernel from preferred session/Compare Sal..* - 

## Read in all CTD profile data

- Excluding upcasts (`*up*` in filename)
- Excluding casts that were not binned (`*_nobinavg*` in filename)
- Excluding casts with `*restart*` in filename (not sure what these are?)
- Excluding St2-C1, I think we know this is bad

#### Imports

In [13]:
import seabird

 This is a seabird parser found here: https://github.com/castelao/seabird 
 It looks pretty good - but yields some annoying warnings and does 
 interfacte directly with xarray, and does not convert to nice names etc.

 -> Working on a parser here (https://github.com/NPIOcean/oceanograPy/blob/master/io/cnv.py) - *not done yet*

Suppress warnings from seabird.fCNV (need to look into later, terribly annoying now).

Warnings are on the form:

     scan was supposed to have 1197 values, but found only 1196.
     prdM was supposed to have 1197 values, but found only 1196.
     t4990C was supposed to have 1197 values, but found only 1196.

(set `'disable_existing_loggers': False,` to recover the warnings).

In [14]:
import logging.config
logging.config.dictConfig({'version': 1,'disable_existing_loggers': True,})

In [15]:
# For mapping
import cartopy.crs as ccrs
import cartopy.feature

Make plots pop out into windows rather than plot inline within the notebook:

In [16]:
%matplotlib tk

### Load data

In [17]:
basedir = '../data/'
cnvdir = f'{basedir}source/CTD/Processed/'

flist_cnv_all = glob2.glob(f'{cnvdir}*.cnv')
flist_cnv = []
for fn in flist_cnv_all:
    if 'up' not in fn:
        if '_nobinavg' not in fn:
            if 'restart' not in fn:
                flist_cnv += [fn]

# Confirmed bad files (no data, etc)
bad_casts = ['sta002_01.cnv']

for bad_cast in bad_casts:
    for fn in flist_cnv:
        if bad_cast in fn:
            flist_cnv.remove(fn)

flist_cnv = list(np.sort(flist_cnv)) # <- List of source .cnv files

In [18]:
def get_CTDnum_from_cnv_name(cnv):
    '''
    Takes the file name, returns the ctd station.
    '''
    sta_ind = fn.rfind('sta') + 3
    uscore_ind = fn.rfind('_')
    station = int(fn[sta_ind:uscore_ind])
    return station

Loop through and read 
    
- Profiles (into `D`)
- S values at certain depths (`dep0, .., dep3`)


In [19]:
# NOTE: ugly code - refactor later

dep0, dep1, dep2, dep3, dep4 = 15, 500, 1000, 2000, 3000
S_dep0, S_dep1, S_dep2, S_dep3, S_dep4 = [], [], [], [], []
stations = []
cnvnames = []

D = []

for fn in flist_cnv:
    try:
        d = seabird.cnv.fCNV(fn).as_DataFrame()
        if 'PRES' not in d.columns:
            d['PRES'] = d.prdM
            
        D += [d]

        s_dep0, s_dep1, s_dep2, s_dep3, s_dep4 = [np.nan]*5

        if d.PRES.max()>dep0:
            s_dep0 = d.loc[d.PRES==dep0].PSAL.values[0]
        if d.PRES.max()>dep1:
            s_dep1 = d.loc[d.PRES==dep1].PSAL.values[0]
        if d.PRES.max()>dep2:
            s_dep2 = d.loc[d.PRES==dep2].PSAL.values[0]
        if d.PRES.max()>dep3:
            s_dep3  = d.loc[d.PRES==dep3].PSAL.values[0]
        if d.PRES.max()>dep4:
            s_dep4  = d.loc[d.PRES==dep4].PSAL.values[0]
            
        S_dep0 += [s_dep0]
        S_dep1 += [s_dep1]
        S_dep2 += [s_dep2]
        S_dep3 += [s_dep3]
        S_dep4 += [s_dep4]

        station = get_CTDnum_from_cnv_name(fn)
        stations += [station]
        cnvnames += [fn.replace(cnvdir, '')]
    except:
        print(f'Could not parse file: {fn} (SKIPPING)')

Could not parse file: ../data/source/CTD/Processed\sta022_02.cnv (SKIPPING)


### Quick analysis by CTD station

Plot the following by CTD station number:

1) S-difference (salinometer minus bottle CTD)
2) Salinity at select depths

In [ ]:
import matplotlib as mpl

In [20]:
p_max, p_min = DF_deep.p_CTD.max(), DF_deep.p_CTD.min()

cmap = mpl.colormaps.get('Spectral').copy()
#cmap.set_extremes(under = p_min, over = p_max)

NameError: name 'mpl' is not defined

In [ ]:
fig, ax = plt.subplots(2, 1, sharex = True)
ax[0].set_facecolor('lightgrey')
mkws = {'marker':'.', 'ms':10, 'alpha':0.7}

for CTD_station in np.arange(1, 22):
    station_salinom = DF_deep.loc[DF_deep.CTD_station ==CTD_station]
    
    for sdiff, p_ in zip(station_salinom.S_diff, station_salinom.p_CTD):
        dcol = cmap((p_ - p_min)/(p_max-p_min))
        ax[0].plot(CTD_station, sdiff, **mkws, color = dcol)
    #for S_CTD, S in station_salinom.S_diff:

# Dummy plot to make colorbar
cb_ax = fig.add_axes([0.72, 0.65, 0.03, 0.2])
xl, yl = ax[0].get_xlim(), ax[0].get_ylim()
C_dummy = ax[0].pcolormesh([-1e3, -1.1e3], [0, 1], [[0]*2]*2, cmap = cmap, vmin = p_min, vmax = p_max)
ax[0].set_xlim(xl) ; ax[0].set_ylim(yl)
cb = plt.colorbar(C_dummy, cax = cb_ax, label = 'Sample\ndepth [db]')
cb_ax.invert_yaxis()

for Sdep, dep in zip([S_dep1, S_dep2, S_dep3, S_dep4],
                     [dep1, dep2, dep3, dep4]):
    ax[1].plot(stations, Sdep, '-k', alpha = 0.3,lw = 0.5)
    
    ax[1].plot(stations, Sdep, **mkws, label = f'{dep} m')

ax[1].legend(title = 'Depth', ncol = 2, handlelength = 1)
for axn in ax:
    axn.grid()

ax[1].set_xlabel('CTD station')
ax[0].set_ylabel('$S_{diff} \  (S_{salinom} - S_{CTD})$')
ax[1].set_ylabel('$S_{CTD}$ [particular depth]')
ax[1].set_xticks(np.arange(1, 23));
ax[0].tick_params(labelbottom = True)

In [21]:
plt.close('all')

### Look at individual profiles

In [22]:
def map_fig():
    fig = plt.figure(figsize=[10, 5])
    ax = plt.subplot(projection=ccrs.SouthPolarStereo())    
    ax.add_feature(cartopy.feature.LAND)
    ax.add_feature(cartopy.feature.OCEAN)
    return(ax)

In [23]:
fig, axs = plt.subplots(1, 3, figsize = (15,6) )
map_ax = map_fig()

ax0, ax1, ax2 = axs
single_out_station = 11

for fn in flist_cnv:
    station = get_CTDnum_from_cnv_name(fn)

    try:
        d = seabird.cnv.fCNV(fn).as_DataFrame()
        if 'PRES' not in d.columns:
            d['PRES'] = d.prdM
        if 'TEMP' not in d.columns:
            d['TEMP'] = d.t4990C
    except: pass
    ax0.plot(d.PSAL, d.PRES, '-k', alpha = 0.3, lw = 0.5)
    ax1.plot(d.TEMP, d.PRES, '-k', alpha = 0.3, lw = 0.5)
    ax2.plot(d.PSAL, d.TEMP, '.k', ms = 1, alpha = 0.3, lw = 0.5)
    
    if station == single_out_station:
       st_kws = {'color':'r', 'alpha':0.9, 'lw':2, 'zorder':3, 'label':f'St #{station}'}
       ax0.plot(d.PSAL, d.PRES, **st_kws)
       ax1.plot(d.TEMP, d.PRES, **st_kws)
       ax2.plot(d.PSAL, d.TEMP, **st_kws)


    # Map
    map_ax.plot(d.LONGITUDE, d.LATITUDE, '.k', transform = ccrs.PlateCarree())
    if station == single_out_station:
        map_ax.plot(d.LONGITUDE, d.LATITUDE, '*r', transform = ccrs.PlateCarree(), zorder = 3,
                   label=f'St #{station}')
        
# Btl and salinometer
st_salinom = DF_deep.loc[DF_deep.CTD_station ==single_out_station]
ax0.plot(st_salinom.S_CTD, st_salinom.p_CTD, '*y', mec = 'k', label = '.btl',  alpha = 0.9, lw = 0.8, zorder = 4)
ax0.plot(st_salinom.S_final, st_salinom.p_CTD, 'sr', mec = 'k',  ms = 3, label = 'salinom', alpha = 0.9, lw = 0.8, zorder = 4)


for axn in [ax0, ax2]:
    axn.set_xlim(34, 35)
for axn in [ax0, ax1]:
    axn.invert_yaxis()
    axn.set_ylim(2200, 0)

for axn in axs: axn.legend()
ax0.set_ylabel('Pressure [db]')


ax0.set_title('S profile')
ax1.set_title('T profile')
ax2.set_title('T-S')

map_ax.set_extent([-20, 40, -60, -75])
map_ax.legend()

## map


C:\Users\hattermann\AppData\Local\miniforge3\envs\TT2021\Lib\site-packages\cartopy\io\__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/50m_physical/ne_50m_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
C:\Users\hattermann\AppData\Local\miniforge3\envs\TT2021\Lib\site-packages\cartopy\io\__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/50m_physical/ne_50m_ocean.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)


In [26]:
fig, ax = plt.subplots()
ax.plot(S_dep1, '-')